# Capstone Project - Segmenting and Clustering Neighborhoods in Toronto

In [17]:
!pip install beautifulsoup4
!pip3 install wikipedia
!apt-get install python3-bs4 (for Python 3)
import requests as rq #library to handle requests
import pandas as pd
import numpy as np
import wikipedia as wiki
from bs4 import BeautifulSoup #library to parse html doc

print('download completed!')

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `apt-get install python3-bs4 (for Python 3)'
download completed!


### Get url via "request librarys"

In [12]:

urk_wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table_class = "wikitable sortable jquery-tablesorter"
response = rq.get(urk_wiki)
print(response.status_code) # check if it shows '200', so it's legal to use data

200


In [13]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')



In [14]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_data = df
toronto_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Dataframe shape

In [15]:
toronto_data.shape

(103, 3)